<a href="https://colab.research.google.com/github/hansong0219/Pytorch_Notebooks/blob/main/CNN_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

cpu


In [3]:
EPOCHS = 40
BATCH_SIZE = 64

In [4]:
train_loader = torch.utils.data.DataLoader(datasets.FashionMNIST('D:/#.Secure Work Folder/hs_dataset/',
                                                                train = True,
                                                                download = True,
                                                                transform = transforms.Compose([
                                                                    transforms.ToTensor(),
                                                                    transforms.Normalize((0.1307,),(0.3081,))
                                                                ])),
                                          batch_size = BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(datasets.FashionMNIST('D:/#.Secure Work Folder/hs_dataset/',
                                                                train = False,
                                                                download = True,
                                                                transform = transforms.Compose([
                                                                    transforms.ToTensor(),
                                                                    transforms.Normalize((0.1307,),(0.3081,))
                                                                ])),
                                          batch_size = BATCH_SIZE, shuffle=True)


Extracting D:/#.Secure Work Folder/hs_dataset/FashionMNIST/raw/train-images-idx3-ubyte.gz to D:/#.Secure Work Folder/hs_dataset/FashionMNIST/raw




Extracting D:/#.Secure Work Folder/hs_dataset/FashionMNIST/raw/train-labels-idx1-ubyte.gz to D:/#.Secure Work Folder/hs_dataset/FashionMNIST/raw




Extracting D:/#.Secure Work Folder/hs_dataset/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to D:/#.Secure Work Folder/hs_dataset/FashionMNIST/raw




Extracting D:/#.Secure Work Folder/hs_dataset/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to D:/#.Secure Work Folder/hs_dataset/FashionMNIST/raw

Processing...


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size = 5) # 입력층의 채널수, 필터수, 커넬 크기
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 5)
        self.drop = nn.Dropout2d() # Convnet 을 위한 dropout 층
        
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320) # TF 에서 Flatten - Dense의 역할을 함
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1) 

In [6]:
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

In [7]:
def train(model, train_loader, optimizer, epoch):
    model.train() #모델을 Train 상태로 전환
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad() 
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx%200==0:
            print('train Epoch: {}[{}/{} ({:.0f}%)]\tLoss:{:.6f}'
                  .format(epoch, batch_idx*len(data),len(train_loader.dataset),
                          100.*batch_idx/len(train_loader),loss.item()))
            
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            # 배치의 오차를 합산
            test_loss += F.cross_entropy(output,target,reduction='sum').item()
            
            #예측 값을 추출
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100.* correct/len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

train Epoch: 1[0/60000 (0%)]	Loss:2.276202
train Epoch: 1[12800/60000 (21%)]	Loss:1.178749
train Epoch: 1[25600/60000 (43%)]	Loss:0.963314
train Epoch: 1[38400/60000 (64%)]	Loss:0.911726
